In [139]:
%load_ext autoreload
%autoreload 2

from grid import load
from functools import partial 

from stability import *
from data import *
from laplacian import *
from perturbations import *
from utils import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'matplotlib'

In [131]:
prefix = '/home/lpetrini/results/'
filename = 'twopoints22'

In [132]:
runs = load(prefix + filename + '_stab')

In [133]:
df = pd.DataFrame()

for r in runs:
    df = pd.concat([df,
        r['df']
    ], )

In [134]:
df = df[(df.net == 'VGG11bn') | (df.net == 'VGG11bn') | (df.net == 'VGG16bn')]

In [135]:
df['ptr'] = df.run_args.apply(lambda a: a.ptr)

df['sch'] = df.args.apply(lambda a: a.shuffle_channels)

In [136]:
df['R'] = df['D'] / df['G']
groups = ['ptr', 'sch', 'net', 'trained', 'layer']

R = df.groupby(groups)['R'].apply(np.mean)
D = df.groupby(groups)['D'].apply(np.mean)
G = df.groupby(groups)['G'].apply(np.mean)

In [138]:
%matplotlib notebook
gs = gridspec_kw={'height_ratios': [2.3, 1]}
fig, ax = plt.subplots(1, 1, figsize=(5.5, 2.1), dpi=150, sharex=True)

tr = 1
nets = df.net.unique()
nets.sort()

for i, net in enumerate(ng):
    for sch in [0, 1]:
        r = R[ptr][sch][net][tr].values # / R[net][0].values
        plt.plot(np.linspace(0, 1, len(r)), r, c=f'C{i}', ls='--' if sch else '-', label=net if not sch else None)

plt.ylabel('$R_f$', fontsize=10)
plt.yscale('log')

plt.plot([], [], 'k', label='original')
plt.plot([], [], '--k', label='shuffle ch.')
#     plt.plot([], [], 'k', ls='dotted', label='ratio')
plt.legend(fontsize=8, ncol=2)
plt.xlim(0, 1)
plt.xlabel('relative depth', fontsize=9)
plt.annotate(f'TwoPixelsDataset', xy=(0.6, 0.85), xycoords='axes fraction', horizontalalignment='left', fontsize=15)

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
%matplotlib notebook
fig, ax = plt.subplots(1, 1, figsize=(9, 4), dpi=150)

ng = ['VGG11', 'VGG11bn', 'VGG16bn']
ptr, sch = 1024, 2, 'L2', 14, 1

ni = -1
for net in ng:
#     if 'bn' not in net: continue
    ni += 1
    print(i)
    for tr in [1]:
        r = R[ptr][sch][gap][norm][net][tr].values
        if 'aVGG' in net:
            idxs = np.asarray([ll in ['M', 'Relu', 'A'] for ll in layer_names[net]])
        else:
            idxs = np.asarray([True for _ in r])
        plt.plot(np.linspace(0, 1, sum(idxs)), r[idxs], c=f'C{ni}', ls='--' if not tr else '-', marker='.', label=net if tr else None)
        for i, v in enumerate(zip(np.linspace(0, 1, sum(idxs[:-1])), r[idxs][:-1])):
            plt.text(*v, layer_names[net][i], fontsize=5)

plt.xlabel('relative depth', fontsize=11)
plt.ylabel('$R_f$', fontsize=15)

plt.plot([], [], '--k', label='init.')
plt.plot([], [], 'k', label='trained')
plt.legend(fontsize=8)

# plt.yscale('log')

plt.xlim(0, 1)
plt.xticks(np.linspace(0, 1, 5), np.linspace(0, 1, 5)) #[r'$0$', r'$\frac{1}{4}$', r'$\frac{1}{2}$', r'$\frac{3}{4}$', r'1']
plt.title(fr'TwoPointsDataset | shuffle channels | $P = {ptr:.00f}, d = 28, \xi = {int(xi)}$, dist={norm}, gap={gap}')
# plt.title(fr'TwoPointsDataset | $P = {ptr:.00f}, d = 28, \xi = {int(xi)}$, dist={norm}, gap={gap}')
plt.yscale('log')
plt.tight_layout()

# plt.savefig('./figures/R_f_depth.pdf', format='pdf')